# Process optimization by genetic algorithm

Smaller size version

In [108]:
include("../src/ProcGA.jl")

Main.ProcGA

## reading data

In [2]:
using CSV
using DelimitedFiles
using DataFrames
using Missings

In [3]:
ptbl = CSV.read("proctable.csv",header = 1);

In [61]:
# take smaller size 
ptbl = ptbl[1:6,:];

In [62]:
mtbl = convert(Array, ptbl[:,1:2])

6×2 Array{Any,2}:
 "BB105 間管"       50
 "BB201-3 3補助管2"  50
 "BH301M 間管"      50
 "BH301M 枝管1"     50
 "BH301M 枝管2"     50
 "EP201 間管"       50

In [63]:
ptbl2 = [coalesce.(x,0) for x in convert(Array, ptbl2)]

6×14 Array{Int64,2}:
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2

In [64]:
s = string.(names(ptbl)[3:end])

14-element Array{String,1}:
 "pipecut"           
 "anneal_bs_prebend" 
 "anneal_ns_prebend" 
 "icing"             
 "rotbend"           
 "pressbend"         
 "anneal_bs_precalib"
 "anneal_ns_precalib"
 "vcalib"            
 "hcalib"            
 "buff_precut"       
 "mc_cut"            
 "buff_postcut"      
 "qc_check"          

In [65]:
nametable = DataFrame(id = 1:length(s), nm = s)

,id,nm
1,1,pipecut
2,2,anneal_bs_prebend
3,3,anneal_ns_prebend
4,4,icing
5,5,rotbend
6,6,pressbend
7,7,anneal_bs_precalib
8,8,anneal_ns_precalib
9,9,vcalib
10,10,hcalib


Read grouping info.

In [66]:
gtbl = readdlm("grouping.csv",',', skipstart = 1)

2×4 Array{Any,2}:
 "anneal_bs"  "[2;7]"  150  3
 "anneal_ns"  "[3;8]"  150  3

In [67]:
gtbl2=[]
for k in 1:size(gtbl)[1]
    x = gtbl[k, 2:end]
    push!(gtbl2,Dict(:id=>eval(Meta.parse(x[1])),:cnt=>x[2],:timespan=>x[3]))
end

In [68]:
gtbl2

2-element Array{Any,1}:
 Dict{Symbol,Any}(:timespan=>3,:id=>[2, 7],:cnt=>150)
 Dict{Symbol,Any}(:timespan=>3,:id=>[3, 8],:cnt=>150)

## Initialize ProcGA tables.

In [109]:
ProcGA.settable(ptbl2, gtbl2, mtbl)

true

## Initialize population

In [70]:
gn1 = ProcGA.jobtableshuffle()

6×72 Array{Int64,2}:
 13   0   0  0  0   0   0  0  0   0  0  …   0  0  0   0   0  0   0   0  0   0
  0   0   0  0  0   0   0  0  0   0  0      0  7  0   0   0  1   0   0  0   9
  7   0   0  1  0   0   0  9  0  12  0     12  0  0   0   0  0   0   0  0   0
  0  13   7  0  0  14   7  0  0   0  0      0  0  0   0   0  0   0   0  9  12
  0   7   0  0  0   0  12  0  0   0  0      0  0  0   0   0  0  12  13  0   0
  0   0  13  0  0   0   0  0  0   0  0  …   0  0  0  12  14  7   0   0  0   0

In [71]:
ProcGA.shrinkjob!(gn1)

6×72 Array{Int64,2}:
 13   0   0  0   0   0  0   0  0  7  0  …  0  0  0  0  0  0  0  0  0  0  0  0
  0   0   0  0   0   0  0   0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
  7   0   0  1   0   0  9  12  0  0  7     0  0  0  0  0  0  0  0  0  0  0  0
  0  13   7  0  14   7  0   0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
  0   7   0  0   0  12  0   0  1  0  7     0  0  0  0  0  0  0  0  0  0  0  0
  0   0  13  0   0   0  0   0  0  0  9  …  0  0  0  0  0  0  0  0  0  0  0  0

In [72]:
ProcGA.validatejob!(gn1,100000)

6×72 Array{Int64,2}:
 1  0  0  0  0  0  0   0   0  7   0  …   0  0   0  0   0  0   0   0  0   0
 0  1  7  7  7  9  9  12  12  0  13      0  0   0  0   0  0   0   0  0   0
 0  0  0  0  1  0  0   0   0  0   0     14  0   0  0  14  0   0   0  0   0
 0  0  1  7  0  0  0   0   7  0   0      0  0   0  0   0  0   0   0  0   0
 0  0  0  0  0  0  0   0   0  0   1     13  0  14  0   0  0   0   0  0  14
 0  0  0  0  0  0  0   1   0  0   0  …   0  0   0  0   0  0  14  14  0   0

In [73]:
ProcGA.checkvalidity(gn1)

true

Make population

In [74]:
ppl = ProcGA.initpopulation(100);

In [ ]:
vlst = []

## Evolution!

Due to slightly large data, validation can not be done completely under max iteration = 100.

In [138]:
# Anyway move generation
vl = ProcGA.evolution!(ppl, 500,20,0.7,0.1,0.2);

i:20 => (34, 34.0, 281)
i:40 => (34, 34.0, 275)
i:60 => (34, 34.0, 262)
i:80 => (34, 34.0, 271)
i:100 => (34, 34.0, 299)
i:120 => (34, 34.0, 288)
i:140 => (34, 34.0, 266)
i:160 => (34, 34.0, 290)
i:180 => (33, 34.0, 258)
i:200 => (33, 34.0, 279)
i:220 => (33, 33.0, 328)
i:240 => (33, 33.0, 295)
i:260 => (33, 33.0, 247)
i:280 => (33, 33.0, 280)
i:300 => (33, 33.0, 334)
i:320 => (33, 33.0, 277)
i:340 => (33, 33.0, 274)
i:360 => (33, 33.0, 277)
i:380 => (33, 33.0, 251)
i:400 => (33, 33.0, 239)
i:420 => (33, 33.0, 305)
i:440 => (33, 33.0, 240)
i:460 => (33, 33.0, 326)
i:480 => (33, 33.0, 285)
i:500 => (33, 33.0, 282)


In [139]:
vlst = vcat(vlst,vl);

In [140]:
v = ProcGA.penalty.(ppl)
idm = findmin(v)[2]
bestgen = ppl[idm]

6×72 Array{Int64,2}:
 0  0  0  1  7  7  7  9  9  12  12  13  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  0  0  0  0  0   9   9   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   0   1   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   1   7   7  …  0  0  0  0  0  0  0  0  0  0  0  0

In [141]:
ProcGA.checkvalidity(bestgen)

true

In [142]:
ProcGA.penalty(bestgen)

33

In [143]:
ProcGA.clipjob(bestgen)

6×33 Array{Int64,2}:
 0  0  0  1  7  7  7  9  9  12  12  13  …   0   0   0   0   0   0   0   0   0
 1  7  7  7  0  0  0  0  0   9   9   0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0   0   1   0     12  12   0  13  13  14  14   0   0
 0  1  0  0  7  7  7  0  0   0   0   0     13   0   0  14  14   0   0   0   0
 0  0  0  0  0  1  0  7  7   7   0   9     14  14   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0   1   7   7  …   0  13  13   0   0   0   0  14  14

In [144]:
# save best process table to file.
# do not clip it so that it can be used again
writedlm("bestprc_2081112.csv",bestgen, ',')

## check